In [6]:
import numpy as np
import pandas as pd
np.set_printoptions(precision=4)

# Решение задачи Дирихле:
## Известно, что $u(x,y) = 2x^3y^3$
## Оно является решением задачи Дирихле:
### $\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} = -(\underbrace{-12xy(x^2+y^2)}_{f(x,y)})$
### $u(0,y) \equiv 0$
### $u(1,y) = 2y^3$
### $u(x,0) \equiv 0$
### $u(x,1) = 2x^3$

In [15]:
N = 5
a_x = 0
b_x = 1
a_y = 0
b_y = 1
h = (b_x-a_x)/(N - 1) # ширина сетки вместе с граничными узлами
iN = N - 2 # ширина сетки без граничных узлов

f = lambda x,y: -12*x*y*(x*x+y*y)
u1y = lambda y: 2*y**3
ux1 = lambda x: 2*x**3
u_expected = lambda x,y: 2*x**3*y**3

### Задаём начальное приближение с помощью начальных данных

In [ ]:
def init_U():
    U = np.zeros(shape = (N, N))
    for i in range(N):
        U[i, N - 1] = ux1(i*h)
    for j in range(N):
        U[N - 1, j] = u1y(j*h)
    return U

U = init_U()

U_expected = U.copy() # задаём границы
U_expected[1:iN+1,1:iN+1] = np.array([[u_expected(i*h,j*h) for j in range(1, iN+1)] for i in range(1, iN+1)])

F = np.array([[f(i*h, j*h) for j in range(1, iN+1)] for i in range(1,iN+1)])

In [17]:
class DESolver:
    def __init__(self, grid_step, N, eps = 1e-5):
        self.h = grid_step
        self.methods = {'iteration': self.iteration_method_iter,
                        'seidel': self.seidel_method_iter,
                        'over-relaxation': self.overrelaxation_method_iter}
        self.methods_params = {'iteration'      : {'min_iters': 2*np.log(1/eps)/(np.pi*h)**2,
                                                'spectral_rad': np.cos(np.pi * h)},
                               'seidel'         : {'min_iters': np.log(1/eps)/(np.pi*h)**2,
                                                'spectral_rad': np.cos(np.pi * h)**2},
                               'over-relaxation': {'min_iters': 2*np.log(1/eps)/(np.pi*h),
                                                'spectral_rad': np.cos(np.pi * h),
                                                'opt_parameter': 2/(1 + np.sin(np.pi * h))}}
        self.iN = N - 2
        C = np.matrix([[4, -1] + (N-2)*[0]] +
              [ k*[0] + [-1, 4, -1] + (N - 3 - k)*[0] for k in range(0, N - 2)] +
              [(N-2)*[0] + [-1, 4]], dtype=float)
        self.mmatrix = np.block([[C, -np.eye(N = N)] + (N-2)*[np.zeros( shape = (N, N) )]] +
              [ k*[np.zeros( shape = (N, N) )] + [-np.eye(N = N), C, -np.eye(N = N)] + (N - 3 - k)*[np.zeros( shape = (N, N) )] for k in range(0, N - 2)] +
              [(N-2)*[np.zeros( shape = (N, N) )] + [-np.eye(N = N), C]]) / grid_step**2
        self.lastsol = np.zeros(shape = (N, N))
        self.iter_diffs = np.zeros(shape = 3) # Хранит разницу приближений последних трёх итераций

    def __method_iter(self, U_curr, F, method_name, method_params):
        return self.methods[method_name](U_curr, F, params = method_params)

    def iteration_method_iter(self, U, F, params = None):
        for i in range(1, self.iN+1):
            for j in range(1, self.iN+1):
               self.lastsol[i, j] = 0.25*(U[i-1, j]+U[i+1,j]+U[i, j-1]+U[i, j+1]+F[i-1,j-1]*self.h**2)
        return self.lastsol

    def seidel_method_iter(self, U, F,params = None):
        for k in range(1, iN+1):
            for i in range(k, iN+1):
                self.lastsol[i, k] = 0.25*(self.lastsol[i-1, k]+U[i+1,k]+self.lastsol[i, k-1]+U[i, k+1]+F[i-1,k-1]*self.h**2)
            for j in range(k+1, iN+1):
                self.lastsol[k, j] = 0.25*(self.lastsol[k-1, j]+U[k+1,j]+self.lastsol[k, j-1]+U[k, j+1]+F[k-1,j-1]*self.h**2)
        return self.lastsol

    def overrelaxation_method_iter(self, U, F,params):
            for k in range(1, iN+1):
                for i in range(k, iN+1):
                    self.lastsol[i, k] = (1-params['opt'])*U[i,k] + 0.25*params['opt']*(self.lastsol[i-1, k]+U[i+1,k]+self.lastsol[i, k-1]+U[i, k+1]+F[i-1,k-1]*self.h**2)
                for j in range(k+1, iN+1):
                    self.lastsol[k, j] = (1-params['opt'])*U[k,j] + 0.25*params['opt']*(self.lastsol[k-1, j]+U[k+1,j]+self.lastsol[k, j-1]+U[k, j+1]+F[k-1,j-1]*self.h**2)
            return self.lastsol

    def __calculate_props(self, U, U_expected, F, iter, init_error, init_discrepancy, apost_est_coef):
        iter_diff = np.max(np.abs(U - self.lastsol))
        self.iter_diffs[iter % 3] = iter_diff
        apost_est = apost_est_coef * iter_diff
        spec_rad_est = ' ' if iter < 2 else np.sqrt(self.iter_diffs[iter % 3]/ self.iter_diffs[(iter - 2) % 3] )

        U_reshaped = np.reshape(self.lastsol, newshape=(N**2, 1)) # преобразуем матрицу сетки к вектору
        AU = self.mmatrix @ U_reshaped
        AU_reshaped_cut = np.reshape(AU, newshape=(N, N))[1:iN+1, 1:iN+1]

        error = np.max(np.abs(self.lastsol - U_expected))
        discrepancy = np.max(np.abs(AU_reshaped_cut - F))
        rel_discrepancy = discrepancy / init_discrepancy
        rel_error = error / init_error

        return [iter + 1, discrepancy, rel_discrepancy,
                             error, rel_error, iter_diff, apost_est, spec_rad_est]

    def solve(self, U_init, U_expected, F, eps = 1e-5, method = 'iteration', method_parameters = None):
        if method not in ['iteration', 'seidel', 'over-relaxation']:
            return None
        else:
            self.iter_diffs = np.zeros(shape = 3)
            if method_parameters is None and method == 'over-relaxation':
                method_parameters = {'opt': self.methods_params['over-relaxation']['opt_parameter']}
        apost_est_coef = self.methods_params[method]['spectral_rad']/ (1 - self.methods_params[method]['spectral_rad'])
        init_error = np.max(np.abs(U_init - U_expected))
        U = U_init.copy()
        self.lastsol = U.copy()

        U_reshaped = np.reshape(U, newshape=(N**2, 1)) # преобразуем матрицу сетки к вектору
        AU = self.mmatrix@U_reshaped
        AU_reshaped_cut = np.reshape(AU, newshape=(N, N))[1:iN+1, 1:iN+1]

        init_discrepancy = np.max(np.abs(AU_reshaped_cut - F))
        output_list = list()
        iter = 0
        rel_discrepancy = 1

        while rel_discrepancy > eps and iter < self.methods_params[method]['min_iters']:
            U_new = self.__method_iter(U, F, method, method_parameters)

            curr_iter_props = self.__calculate_props(U, U_expected, F, iter, init_error, init_discrepancy, apost_est_coef)
            output_list+=[ curr_iter_props ]

            U = U_new.copy()
            rel_discrepancy = curr_iter_props[2]
            iter+=1
                                                                                      
        output_list += [[' ', 'min_iters:', self.methods_params[method]['min_iters'], ' ', ' method:', method, ' ', ' ']]
        return {'solution': U ,'output': pd.DataFrame(data = output_list,
                                                      columns = ['k', 'Curr. discr.', 'Rel. discr.', 'Curr. error', 'Rel. error', '$ \lVert U_k - U_{k-1} \rVert $', 'Apost. est.', '$\hat{\rho}_k$']).set_index(keys = 'k')}


In [18]:
solver = DESolver(h, N)

### Решение методом простой итерации

In [19]:
iter_summary = solver.solve(U, method = 'iteration', U_expected = U_expected, F = F)
solU_i = iter_summary['solution']
iter_summary['output'][::]

,Curr. discr.,Rel. discr.,Curr. error,Rel. error,$ \lVert U_k - U_{k-1} \rVert $,Apost. est.,$\hat{\rho}_k$
k,,,,,,,
1,4.25,0.219002,0.100098,0.281207,0.303223,0.732044,
2,2.125,0.109501,0.055664,0.156379,0.066406,0.160319,
3,1.210938,0.062399,0.033386,0.093793,0.033203,0.080159,0.330909
4,0.90625,0.046699,0.027832,0.078189,0.018921,0.045679,0.533785
5,0.491211,0.025312,0.01461,0.041045,0.01416,0.034186,0.653047
6,0.453125,0.023349,0.013916,0.039095,0.007675,0.01853,0.636903
7,0.231323,0.011920,0.007045,0.019791,0.00708,0.017093,0.707107
8,0.226562,0.011675,0.006958,0.019547,0.003614,0.008726,0.686239
9,0.113876,0.005868,0.00349,0.009804,0.00354,0.008546,0.707107


### Решение методом Зейделя

In [20]:
iter_summary = solver.solve(U, method = 'seidel', U_expected = U_expected, F = F)
solU_s = iter_summary['solution']
iter_summary['output'][::]

,Curr. discr.,Rel. discr.,Curr. error,Rel. error,$ \lVert U_k - U_{k-1} \rVert $,Apost. est.,$\hat{\rho}_k$
k,,,,,,,
1,4.8078,0.247745,0.110939,0.311664,0.300488,0.300488,
2,2.300262,0.118532,0.067169,0.1887,0.071883,0.071883,
3,0.80603,0.041535,0.033585,0.09435,0.033585,0.033585,0.334316
4,0.403015,0.020767,0.016792,0.047175,0.016792,0.016792,0.483327
5,0.249527,0.012858,0.008396,0.023588,0.008396,0.008396,0.5
6,0.131945,0.006799,0.004198,0.011794,0.004198,0.004198,0.5
7,0.06687,0.003446,0.002099,0.005897,0.002099,0.002099,0.5
8,0.033547,0.001729,0.00105,0.002948,0.00105,0.00105,0.5
9,0.016788,0.000865,0.000525,0.001474,0.000525,0.000525,0.5


### Решение методом верхней релаксации

In [21]:
iter_summary = solver.solve(U, method = 'over-relaxation', U_expected = U_expected, F = F)
solU_or = iter_summary['solution']
iter_summary['output'][::]

,Curr. discr.,Rel. discr.,Curr. error,Rel. error,$ \lVert U_k - U_{k-1} \rVert $,Apost. est.,$\hat{\rho}_k$
k,,,,,,,
1,5.683959,0.292893,0.114415,0.32143,0.350007,0.844991,
2,3.294385,0.169759,0.072523,0.20374,0.100462,0.242537,
3,1.430378,0.073707,0.031478,0.088431,0.058353,0.140876,0.408313
4,0.837896,0.043177,0.015824,0.044454,0.026015,0.062805,0.508873
5,0.041904,0.002159,0.001068,0.003001,0.015338,0.03703,0.512695
6,0.024547,0.001265,0.000542,0.001524,0.00075,0.001811,0.169828
7,0.004205,0.000217,0.000093,0.000262,0.000449,0.001085,0.171159
8,0.000711,0.000037,0.000016,0.000045,0.000077,0.000186,0.320297
9,0.000123,0.000006,0.000003,0.000009,0.000013,0.000031,0.17025
